In [7]:
%config IPCompleter.greedy=True
import numpy as np
import pandas as pd
import psycopg2
import matplotlib.pyplot as plt
import time
import re
import scipy
from datetime import time, timedelta
import seaborn as sns
import datetime as dt
pd.set_option('display.max_columns', 300)

In [8]:
def trans_id(df):
    for i in ['subject_id','hadm_id','stay_id']:
        if i in df.columns.tolist():
            if df[i].dtype == object:
                if ((df[i]== 'None')|(df[i]== 'Null')).any():
                    df[i].replace('None',np.nan,inplace = True)
                    df[i].replace('Null',np.nan,inplace = True)
                    df[i] = df[i].astype(float)
                else:
                    df[i] = df[i].astype(float)

In [9]:
def trans_time(t):
    if t == 'None':
        return np.datetime64('NaT')
    elif pd.isna(t):
        return np.datetime64('NaT')
    else:
        return np.datetime64(t)

In [10]:
def trans_float(df):
    for i in df.columns:
        if df[i].dtype == object:
            if ((df[i]== 'None')|(df[i]== 'Null')).any():
                df[i].replace('None',np.nan,inplace = True)
                df[i].replace('Null',np.nan,inplace = True)
                df[i] = df[i].astype(float)
            else:
                df[i] = df[i].astype(float)

In [98]:
data = pd.read_csv('data_tomodel_1st_revise.csv')

In [99]:
data = data.drop_duplicates()

In [93]:
#from pandas_profiling import ProfileReport

In [94]:
#profile = ProfileReport(data, title="Pandas Profiling Report", minimal=True, pool_size=0)

In [95]:
#profile.to_file("ini_1st_eda.html")

In [96]:
variables_name = [
 'stay_id',
 'gender',
# 'admittime',
# 'dischtime',
# 'los_hospital',
 'admission_age',
 'race',
 'height',
 'weight',
 'before_adm_90d',

 'history_unintentional_weight_loss_10_lbs',
 'history_assistive_devices_Cane',
 'history_assistive_devices_None',
 'history_assistive_devices_Other',
 'history_assistive_devices_Walker',
 'history_assistive_devices_Wheelchair',
 'history_currently_pain',
 'history_dialysis',
 'history_difficulty_swallowing',
 'history_iv_access',
 'history_past_Anemia',
 'history_past_Asthma',
 'history_past_COPD',
 'history_past_Diabetes - Insulin',
 'history_past_Diabetes - Oral Agent',
 'history_past_ETOH',
 'history_past_GI Bleed',
 'history_past_HEMO or PD',
 'history_past_Hepatitis',
 'history_past_Liver Failure',
 'history_past_Pancreatitis',
 'history_past_Renal Failure',
 'history_past_Seizures',
 'history_past_Smoker',
 'history_cv_Angina',
 'history_cv_Arrhythmias',
 'history_cv_CAD',
 'history_cv_CHF',
 'history_cv_CVA',
 'history_cv_Hypertension',
 'history_cv_MI',
 'history_cv_PVD',
 'history_cv_Pacemaker',   

 'myocardial_infarct',
 'congestive_heart_failure',
 'peripheral_vascular_disease',
 'cerebrovascular_disease',
 'dementia',
 'chronic_pulmonary_disease',
 'rheumatic_disease',
 'peptic_ulcer_disease',
 'mild_liver_disease',
 'diabetes_without_cc',
 'diabetes_with_cc',
 'paraplegia',
 'renal_disease',
 'malignant_cancer',
 'severe_liver_disease',
 'metastatic_solid_tumor',
 'aids',
 'charlson_comorbidity_index',
    

 'nasal_swab',
 'blood_culture',
 'urine_culture',
 'sputum_culture',
 'stool_culture',
 'pan_culture',
 'bal_fluid_culture',
 'wound_culture',
 'rectal_swab',
 'csf_culture',

 'admission_location_AMBULATORY SURGERY TRANSFER',
 'admission_location_CLINIC REFERRAL',
 'admission_location_EMERGENCY ROOM',
 'admission_location_INFORMATION NOT AVAILABLE',
 'admission_location_INTERNAL TRANSFER TO OR FROM PSYCH',
 'admission_location_PACU',
 'admission_location_PHYSICIAN REFERRAL',
 'admission_location_PROCEDURE SITE',
 'admission_location_TRANSFER FROM HOSPITAL',
 'admission_location_TRANSFER FROM SKILLED NURSING FACILITY',
 'admission_location_WALK-IN/SELF REFERRAL',
# 'hospital_expire_flag',

 'hospstay_seq',
 'icustay_rank',
 'los_hosp_beforeicu',
 'previous_mdrpos_lessthan90d',
 'previous_mdrneg_lessthan90d', 
# 'first_hosp_stay',
# 'intime',
# 'outtime',
 'first_careunit',
 'last_careunit',
# 'los_icu',
# 'icustay_seq',
# 'first_icu_stay',
# 'firsthospmdrcharttime',
# 'firsthospmdrstoretime',
# 'firsthospmdrorgname',
# 'firsthospmdrRAB',
# 'last',
# 'previous_mdrpos_lessthan14d',
# 'previous_mdrneg_lessthan14d',

# 'stay_chart_included',
 'immu_duration',
 'ant_duration',
 'ant_1stday',
 'immu_1stday',
 'vaso_beforeicu',
 'vaso_1stday',
 'surgical',
 'invasive_line_1stday',
 'ventilation_1stday',
 'ent_nut',
 'par_nut',

'lods', 'sofa', 'sirs', 'apsiii', 'sapsii', 'oasis',
             
    'albumin', 'globulin', 'total_protein', 'aniongap', 'bicarbonate',
    'bun', 'calcium', 'chloride', 'creatinine', 'glucose', 'sodium',
    'potassium', 'so2', 'po2', 'pco2', 'fio2_chartevents', 'fio2', 'aado2',
    'aado2_calc', 'pao2fio2ratio', 'ph', 'baseexcess', 'totalco2',
    'carboxyhemoglobin', 'methemoglobin', 'lactate', 'specimen_id',
    'basophils_abs', 'eosinophils_abs', 'lymphocytes_abs', 'monocytes_abs',
    'neutrophils_abs', 'basophils', 'eosinophils', 'lymphocytes',
    'monocytes', 'neutrophils', 'atypical_lymphocytes', 'bands',
    'immature_granulocytes', 'metamyelocytes', 'nrbc', 'bleeding_time',
    'd_dimer', 'fibrinogen', 'thrombin', 'inr', 'pt', 'ptt', 'hematocrit',
    'hemoglobin', 'mch', 'mchc', 'mcv', 'platelet', 'rbc', 'rdw', 'wbc',
    'troponin_i', 'troponin_t', 'ck_mb', 'alt', 'alp', 'ast', 'amylase',
    'bilirubin_total', 'bilirubin_direct', 'bilirubin_indirect', 'ck_cpk',
    'ggt', 'ld_ldh', 'heart_rate', 'spo2', 'sbp', 'dbp', 'mbp', 'resp_rate',
    'temperature', 'gcs', 'gcs_motor', 'gcs_verbal', 'gcs_eyes', 'gcs_unable',
    'crp', 'ck', 'ntprobnp', 'mg', 'mpv', 'phosphate',
 
 'amikacin',
 'amoxicill',
 'amoxicillin_clavulanate',
 'ampicillin',
 'ampicillin_sulbactam',
 'azithromycin',
 'aztreonam',
 'cefuroxime_axetil',
 'bactrim',
 'cefazolin',
 'ceftazidime',
 'ceftazidime_avibactam',
 'cefadroxil',
 'cefepime',
 'cefotaxime',
 'ceftaroline',
 'ceftriaxone',
 'cephalexin',
 'chloramphenicol',
 'ciprofloxacin',
 'clarithromycin',
 'clindamycin',
 'dicloxacillin',
 'doxycycline',
 'erythromycin',
 'gentamicin',
 'cefalexin',
 'levofloxacin',
 'linezolid',
 'nitrofurantoin',
 'metronidazole',
 'meropenem',
 'imipenem',
 'ertapenem',
 'minocycline',
 'moxifloxacin',
 'mupirocin',
 'nafcillin',
 'neomycin_polymyxin',
 'ofloxacin',
 'oxacillin',
 'penicillin',
 'piperacillin_tazobactam',
 'ceftolozane_tazobactam',
 'rifampin',
 'sulfadiazine',
 'polymyxin_trimethoprim',
 'sulfameth_trimethoprim',
 'tetracycline',
 'tobramycin',
 'vancomycin',

 'stay_chart_included_mdr',
 'stay_chart_included_mdr_neg_tot',
 'stay_chart_included_mdr_pos_tot'
]

In [101]:
variables_df = data[variables_name]

# 抗菌素整合

In [18]:
anti_name = ['amikacin', 'amoxicill', 'amoxicillin_clavulanate',
 'ampicillin', 'ampicillin_sulbactam',
 'azithromycin', 'aztreonam',
 'cefuroxime_axetil', 'bactrim',
 'cefazolin', 'ceftazidime',
# 'ceftazidime_avibactam', 'cefadroxil',
 'cefepime', #'cefotaxime',
 'ceftaroline', 'ceftriaxone',
 'cephalexin', #'chloramphenicol',
 'ciprofloxacin', 'clarithromycin',
 'clindamycin', 'dicloxacillin',
 'doxycycline', 'erythromycin',
 'gentamicin', #'cefalexin',
 'levofloxacin', 'linezolid',
 'nitrofurantoin', 'metronidazole',
 'meropenem', 'imipenem',
 'ertapenem', 'minocycline',
 'moxifloxacin', 'mupirocin',
 'nafcillin', 'neomycin_polymyxin',
 #'ofloxacin',
#'oxacillin',
 'penicillin', 'piperacillin_tazobactam',
 'ceftolozane_tazobactam', 'rifampin',
 'sulfadiazine',# 'polymyxin_trimethoprim',
 'sulfameth_trimethoprim', 'tetracycline',
 'tobramycin', 'vancomycin']

In [19]:
pd.pivot_table(data = variables_df,values='stay_chart_included_mdr',index = ['cefotaxime'],aggfunc= ['count','mean'])

,count,mean
,stay_chart_included_mdr,stay_chart_included_mdr
cefotaxime,,
0.0,47587,0.079349


In [20]:
ant_lookdf = pd.DataFrame(columns = ['nonuse_count','use_count','non_use_mdr_ratio','use_mdr_ratio'], index = anti_name)
for i in range(len(anti_name)):
    #ant_lookdf.iloc[i,0] = anti_name[i]
    ant_lookdf.iloc[i,0] = pd.pivot_table(data = variables_df,values='stay_chart_included_mdr',index = [anti_name[i]],aggfunc= ['count','mean']).iloc[0,0]
    ant_lookdf.iloc[i,1] = pd.pivot_table(data = variables_df,values='stay_chart_included_mdr',index = [anti_name[i]],aggfunc= ['count','mean']).iloc[1,0]
    ant_lookdf.iloc[i,2] = pd.pivot_table(data = variables_df,values='stay_chart_included_mdr',index = [anti_name[i]],aggfunc= ['count','mean']).iloc[0,1]
    ant_lookdf.iloc[i,3] = pd.pivot_table(data = variables_df,values='stay_chart_included_mdr',index = [anti_name[i]],aggfunc= ['count','mean']).iloc[1,1]

In [21]:
#ant_lookdf.to_csv('ant_1stdayuse.csv')

In [22]:
antsel_ini_df = variables_df[anti_name]

In [23]:
antsel_ini_df['carbapenem'] = antsel_ini_df[['meropenem','imipenem','ertapenem']].sum(axis = 1).apply(lambda x:1 if x>0 else 0)
antsel_ini_df['ceph_4th'] = antsel_ini_df['cefepime']
antsel_ini_df['ceph_3th'] = antsel_ini_df[['ceftazidime','ceftriaxone']].sum(axis = 1).apply(lambda x:1 if x>0 else 0)
antsel_ini_df['ceph_1st'] = antsel_ini_df[['cephalexin','cefazolin']].sum(axis = 1).apply(lambda x:1 if x>0 else 0)

/tmp/ipykernel_2821004/3798773329.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  antsel_ini_df['carbapenem'] = antsel_ini_df[['meropenem','imipenem','ertapenem']].sum(axis = 1).apply(lambda x:1 if x>0 else 0)
/tmp/ipykernel_2821004/3798773329.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  antsel_ini_df['ceph_4th'] = antsel_ini_df['cefepime']
/tmp/ipykernel_2821004/3798773329.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer

In [24]:
antsel_sel_df = antsel_ini_df[['carbapenem','ceph_4th','ceph_3th','ceph_1st','tobramycin','linezolid',\
              'aztreonam','gentamicin','piperacillin_tazobactam','ciprofloxacin',\
              'metronidazole','levofloxacin','vancomycin','sulfameth_trimethoprim',\
              'clindamycin','erythromycin','azithromycin','doxycycline','ampicillin','nafcillin']]

In [25]:
antsel_sel_df = antsel_sel_df.fillna(0)

# 实验室检查和体征整合

In [26]:
labvitalssel_sel_df = variables_df[['albumin', 'globulin', 'total_protein', 'aniongap', 'bicarbonate',
    'bun', 'calcium', 'chloride', 'creatinine', 'glucose', 'sodium',
    'potassium', 'so2', 'po2', 'pco2', 'fio2_chartevents', 'fio2', 'aado2',
    'aado2_calc', 'pao2fio2ratio', 'ph', 'baseexcess', 'totalco2',
    'carboxyhemoglobin', 'methemoglobin', 'lactate', 'specimen_id',
    'basophils_abs', 'eosinophils_abs', 'lymphocytes_abs', 'monocytes_abs',
    'neutrophils_abs', 'basophils', 'eosinophils', 'lymphocytes',
    'monocytes', 'neutrophils', 'atypical_lymphocytes', 'bands',
    'immature_granulocytes', 'metamyelocytes', 'nrbc', 'bleeding_time',
    'd_dimer', 'fibrinogen', 'thrombin', 'inr', 'pt', 'ptt', 'hematocrit',
    'hemoglobin', 'mch', 'mchc', 'mcv', 'platelet', 'rbc', 'rdw', 'wbc',
    'troponin_i', 'troponin_t', 'ck_mb', 'alt', 'alp', 'ast', 'amylase',
    'bilirubin_total', 'bilirubin_direct', 'bilirubin_indirect', 'ck_cpk',
    'ggt', 'ld_ldh', 'heart_rate', 'spo2', 'sbp', 'dbp', 'mbp', 'resp_rate',
    'temperature', 'crp', 'ck', 'ntprobnp', 'mg', 'mpv', 'phosphate']]

# 入院方式

In [27]:
admissiontype = ['admission_location_AMBULATORY SURGERY TRANSFER',
 'admission_location_CLINIC REFERRAL',
 'admission_location_EMERGENCY ROOM',
 'admission_location_INFORMATION NOT AVAILABLE',
 'admission_location_INTERNAL TRANSFER TO OR FROM PSYCH',
# 'admission_location_None',
 'admission_location_PACU',
 'admission_location_PHYSICIAN REFERRAL',
 'admission_location_PROCEDURE SITE',
 'admission_location_TRANSFER FROM HOSPITAL',
 'admission_location_TRANSFER FROM SKILLED NURSING FACILITY',
 'admission_location_WALK-IN/SELF REFERRAL']

In [28]:
admissiontype_lookdf = pd.DataFrame(columns = ['nonuse_count','value_count','non_mdr_ratio','is_mdr_ratio'], index = admissiontype)
for i in range(len(admissiontype)):
    admissiontype_lookdf.iloc[i,0] = pd.pivot_table(data = variables_df,values='stay_chart_included_mdr',index = [admissiontype[i]],aggfunc= ['count','mean']).iloc[0,0]
    admissiontype_lookdf.iloc[i,1] = pd.pivot_table(data = variables_df,values='stay_chart_included_mdr',index = [admissiontype[i]],aggfunc= ['count','mean']).iloc[1,0]
    admissiontype_lookdf.iloc[i,2] = pd.pivot_table(data = variables_df,values='stay_chart_included_mdr',index = [admissiontype[i]],aggfunc= ['count','mean']).iloc[0,1]
    admissiontype_lookdf.iloc[i,3] = pd.pivot_table(data = variables_df,values='stay_chart_included_mdr',index = [admissiontype[i]],aggfunc= ['count','mean']).iloc[1,1]

In [29]:
admissiontype_lookdf

,nonuse_count,value_count,non_mdr_ratio,is_mdr_ratio
admission_location_AMBULATORY SURGERY TRANSFER,47619,33,0.079401,0.030303
admission_location_CLINIC REFERRAL,47119,533,0.079374,0.078799
admission_location_EMERGENCY ROOM,23457,24195,0.064373,0.093904
admission_location_INFORMATION NOT AVAILABLE,47527,125,0.07945,0.048
admission_location_INTERNAL TRANSFER TO OR FROM PSYCH,47645,7,0.079358,0.142857
admission_location_PACU,47410,242,0.079266,0.099174
admission_location_PHYSICIAN REFERRAL,38054,9598,0.087928,0.045426
admission_location_PROCEDURE SITE,46985,667,0.079706,0.055472
admission_location_TRANSFER FROM HOSPITAL,37107,10545,0.081305,0.072546
admission_location_TRANSFER FROM SKILLED NURSING FACILITY,46935,717,0.077682,0.189679


In [30]:
#admissiontype_lookdf

In [31]:
admissiontype_sel_df = variables_df[['admission_location_EMERGENCY ROOM','admission_location_TRANSFER FROM HOSPITAL','admission_location_TRANSFER FROM SKILLED NURSING FACILITY']]

In [32]:
admissiontype_sel_df['admission_location_other'] = admissiontype_sel_df.apply(lambda x:0 if ((x[0]==1) or (x[1]==1) or (x[2]==1)).any() else 1, axis = 1)

/tmp/ipykernel_2821004/3956313325.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  admissiontype_sel_df['admission_location_other'] = admissiontype_sel_df.apply(lambda x:0 if ((x[0]==1) or (x[1]==1) or (x[2]==1)).any() else 1, axis = 1)


# 既往史

In [33]:
history_ = [ 'history_unintentional_weight_loss_10_lbs',
 'history_assistive_devices_Cane',
 'history_assistive_devices_None',
 'history_assistive_devices_Other',
 'history_assistive_devices_Walker',
 'history_assistive_devices_Wheelchair',
 'history_currently_pain',
 'history_dialysis',
 'history_difficulty_swallowing',
 'history_iv_access',

 'history_past_Anemia',
 'history_past_Asthma',
 'history_past_COPD',
 'history_past_Diabetes - Insulin',
 'history_past_Diabetes - Oral Agent',
 'history_past_ETOH',
 'history_past_GI Bleed',
 'history_past_HEMO or PD',
 'history_past_Hepatitis',
 'history_past_Liver Failure',
 'history_past_Pancreatitis',
 'history_past_Renal Failure',
 'history_past_Seizures',
 'history_past_Smoker',
 'history_cv_Angina',
 'history_cv_Arrhythmias',
 'history_cv_CAD',
 'history_cv_CHF',
 'history_cv_CVA',
 'history_cv_Hypertension',
 'history_cv_MI',
 'history_cv_PVD',
 'history_cv_Pacemaker']

In [34]:
history_sel_df = variables_df[history_]

In [35]:
history_lookdf = pd.DataFrame(columns = ['nonuse_count','value_count','non_mdr_ratio','is_mdr_ratio'], index = history_)
for i in range(len(history_)):
    history_lookdf.iloc[i,0] = pd.pivot_table(data = variables_df,values='stay_chart_included_mdr',index = [history_[i]],aggfunc= ['count','mean']).iloc[0,0]
    history_lookdf.iloc[i,1] = pd.pivot_table(data = variables_df,values='stay_chart_included_mdr',index = [history_[i]],aggfunc= ['count','mean']).iloc[1,0]
    history_lookdf.iloc[i,2] = pd.pivot_table(data = variables_df,values='stay_chart_included_mdr',index = [history_[i]],aggfunc= ['count','mean']).iloc[0,1]
    history_lookdf.iloc[i,3] = pd.pivot_table(data = variables_df,values='stay_chart_included_mdr',index = [history_[i]],aggfunc= ['count','mean']).iloc[1,1]

In [36]:
history_lookdf

,nonuse_count,value_count,non_mdr_ratio,is_mdr_ratio
history_unintentional_weight_loss_10_lbs,44712,2940,0.078167,0.097619
history_assistive_devices_Cane,44512,3140,0.079799,0.073248
history_assistive_devices_None,7761,39891,0.125242,0.070442
history_assistive_devices_Other,46943,709,0.078201,0.156559
history_assistive_devices_Walker,44116,3536,0.076163,0.119344
history_assistive_devices_Wheelchair,45907,1745,0.074673,0.202865
history_currently_pain,37595,10057,0.078016,0.084419
history_dialysis,46089,1563,0.0785,0.104926
history_difficulty_swallowing,44263,3389,0.075187,0.133963
history_iv_access,27586,20066,0.067317,0.095933


# 合并症

In [37]:
comb_ = [ 'myocardial_infarct', 'congestive_heart_failure',
 'peripheral_vascular_disease', 'cerebrovascular_disease', 'dementia',
 'chronic_pulmonary_disease', 'rheumatic_disease',
 'peptic_ulcer_disease', 'mild_liver_disease',
 'diabetes_without_cc', 'diabetes_with_cc',
 'paraplegia', 'renal_disease', 'malignant_cancer',
 'severe_liver_disease', 'metastatic_solid_tumor',
 'aids','charlson_comorbidity_index']

In [38]:
comb_lookdf = pd.DataFrame(columns = ['nonuse_count','value_count','non_mdr_ratio','is_mdr_ratio'], index = comb_)
for i in range(len(comb_)):
    comb_lookdf.iloc[i,0] = pd.pivot_table(data = variables_df,values='stay_chart_included_mdr',index = [comb_[i]],aggfunc= ['count','mean']).iloc[0,0]
    comb_lookdf.iloc[i,1] = pd.pivot_table(data = variables_df,values='stay_chart_included_mdr',index = [comb_[i]],aggfunc= ['count','mean']).iloc[1,0]
    comb_lookdf.iloc[i,2] = pd.pivot_table(data = variables_df,values='stay_chart_included_mdr',index = [comb_[i]],aggfunc= ['count','mean']).iloc[0,1]
    comb_lookdf.iloc[i,3] = pd.pivot_table(data = variables_df,values='stay_chart_included_mdr',index = [comb_[i]],aggfunc= ['count','mean']).iloc[1,1]

In [39]:
comb_lookdf

,nonuse_count,value_count,non_mdr_ratio,is_mdr_ratio
myocardial_infarct,39502,8150,0.080224,0.075215
congestive_heart_failure,33215,14437,0.071474,0.097527
peripheral_vascular_disease,41942,5710,0.079157,0.080911
cerebrovascular_disease,40070,7582,0.081507,0.068056
dementia,45888,1764,0.076643,0.150227
chronic_pulmonary_disease,34775,12877,0.071201,0.101421
rheumatic_disease,45903,1749,0.078426,0.104059
peptic_ulcer_disease,46102,1550,0.079476,0.076129
mild_liver_disease,41401,6251,0.076617,0.097584
diabetes_without_cc,36165,11487,0.073469,0.097937


In [40]:
pd.pivot_table(data = variables_df,values='stay_chart_included_mdr_neg_tot',index = ['charlson_comorbidity_index'],aggfunc= ['count','mean'])

,count,mean
,stay_chart_included_mdr_neg_tot,stay_chart_included_mdr_neg_tot
charlson_comorbidity_index,,
0.0,2619,0.014509
1.0,3122,0.014414
2.0,4174,0.021802
3.0,5213,0.025321
4.0,5989,0.032059
5.0,6076,0.039829
6.0,5444,0.036187
7.0,4668,0.040917


In [41]:
comb_sel_df = variables_df[comb_]

In [42]:
# 'history_cv_MI','myocardial_infarct',
# 'history_cv_CHF', 'congestive_heart_failure',
# 'history_past_COPD','chronic_pulmonary_disease',
# 'diabetes_without_cc', 'diabetes_with_cc', 'history_past_Diabetes - Insulin', 'history_past_Diabetes - Oral Agent',  
# 'history_past_Liver Failure','mild_liver_disease','severe_liver_disease', 'history_past_Hepatitis',
# 'history_cv_Hypertension',
# 'history_cv_CVA','cerebrovascular_disease',

In [43]:
#  'history_past_Anemia',
#  'history_past_Asthma',

#  'history_past_ETOH',
#  'history_past_GI Bleed',
#  'history_past_HEMO or PD',

#  'history_past_Pancreatitis',
#  'history_past_Renal Failure',
#  'history_past_Seizures',
#  'history_past_Smoker',
#  'history_cv_Angina',
#  'history_cv_Arrhythmias',
#  'history_cv_CAD',

#  'history_cv_PVD',
#  'history_cv_Pacemaker'

#  'peripheral_vascular_disease',  'dementia',
#  'rheumatic_disease',
#  'peptic_ulcer_disease', 

#  'paraplegia', 'renal_disease', 'malignant_cancer',
#   'metastatic_solid_tumor',
#  'aids','charlson_comorbidity_index'

# 其他信息

## ICU类型

In [44]:
variables_df['last_careunit'].value_counts()

Medical Intensive Care Unit (MICU)                  11634
Medical/Surgical Intensive Care Unit (MICU/SICU)     8468
Cardiac Vascular Intensive Care Unit (CVICU)         7602
Surgical Intensive Care Unit (SICU)                  7280
Coronary Care Unit (CCU)                             5297
Trauma SICU (TSICU)                                  5075
Neuro Intermediate                                   1166
Neuro Surgical Intensive Care Unit (Neuro SICU)       585
Neuro Stepdown                                        545
Name: last_careunit, dtype: int64

In [45]:
pd.pivot_table(data = variables_df,values='stay_chart_included_mdr',index = ['last_careunit'],aggfunc= ['count','mean'])

,count,mean
,stay_chart_included_mdr,stay_chart_included_mdr
last_careunit,,
Cardiac Vascular Intensive Care Unit (CVICU),7602,0.027493
Coronary Care Unit (CCU),5297,0.058335
Medical Intensive Care Unit (MICU),11634,0.118274
Medical/Surgical Intensive Care Unit (MICU/SICU),8468,0.107936
Neuro Intermediate,1166,0.018868
Neuro Stepdown,545,0.025688
Neuro Surgical Intensive Care Unit (Neuro SICU),585,0.034188
Surgical Intensive Care Unit (SICU),7280,0.075275


## 种族

In [46]:
pd.pivot_table(data = variables_df,values='stay_chart_included_mdr',index = ['race'],aggfunc= ['count','mean'])

,count,mean
,stay_chart_included_mdr,stay_chart_included_mdr
race,,
AMERICAN INDIAN/ALASKA NATIVE,96,0.104167
ASIAN,556,0.082734
ASIAN - ASIAN INDIAN,126,0.103175
ASIAN - CHINESE,496,0.068548
ASIAN - KOREAN,40,0.025000
ASIAN - SOUTH EAST ASIAN,197,0.060914
BLACK/AFRICAN,189,0.042328
BLACK/AFRICAN AMERICAN,4366,0.084746


## 住院次数

In [47]:
pd.pivot_table(data = variables_df,values='stay_chart_included_mdr',index = ['hospstay_seq'],aggfunc= ['count','mean'])

,count,mean
,stay_chart_included_mdr,stay_chart_included_mdr
hospstay_seq,,
1,36338,0.064368
2,6594,0.110252
3,2239,0.126842
4,949,0.150685
5,527,0.184061
6,300,0.190000
7,192,0.203125
8,113,0.168142


In [48]:
pd.pivot_table(data = variables_df,values='stay_chart_included_mdr',index = ['icustay_rank'],aggfunc= ['count','mean'])

,count,mean
,stay_chart_included_mdr,stay_chart_included_mdr
icustay_rank,,
1.0,33407,0.064687
2.0,7820,0.087724
3.0,2944,0.122962
4.0,1352,0.141272
5.0,700,0.155714
6.0,439,0.191344
7.0,254,0.177165
8.0,183,0.185792


In [49]:
pd.pivot_table(data = variables_df,values='stay_chart_included_mdr',index = ['previous_mdrneg_lessthan90d'],aggfunc= ['count','mean'])

,count,mean
,stay_chart_included_mdr,stay_chart_included_mdr
previous_mdrneg_lessthan90d,,
0,46786,0.074424
1,866,0.346420


In [50]:
pd.pivot_table(data = variables_df,values='stay_chart_included_mdr',index = ['gender'],aggfunc= ['count','mean'])

,count,mean
,stay_chart_included_mdr,stay_chart_included_mdr
gender,,
F,20803,0.087535
M,26849,0.073038


# 基础信息表

In [51]:
basic_inf = variables_df[[ 'stay_id',
 'gender',
 'admission_age',
 'race',
 'weight',
 'hospstay_seq',
 'icustay_rank',
 'los_hosp_beforeicu',
 'previous_mdrpos_lessthan90d',
 'previous_mdrneg_lessthan90d', 
 'last_careunit',
 'nasal_swab',
 'blood_culture',
 'urine_culture',
 'sputum_culture',
 'stool_culture',
 'pan_culture',
 'bal_fluid_culture',
 'wound_culture',
 'rectal_swab',
 'csf_culture',
'before_adm_90d'
]]

In [52]:
pd.pivot_table(data = variables_df,values='stay_chart_included_mdr',index = ['stool_culture'],aggfunc= ['count','mean'])

,count,mean
,stay_chart_included_mdr,stay_chart_included_mdr
stool_culture,,
0.0,47144,0.078122
1.0,508,0.194882


# 治疗表

In [53]:
therapy_inf = variables_df[[ 'immu_duration',
 'ant_duration',
 'ant_1stday',
 'immu_1stday',
 'vaso_beforeicu',
 'vaso_1stday',
 'surgical',
 'invasive_line_1stday',
 'ventilation_1stday',
'ent_nut',
 'par_nut']]

# 评分表

In [54]:
scores = variables_df[[ 'gcs',
 'gcs_motor',
 'gcs_eyes',
 'gcs_verbal',
 'gcs_unable',
 'lods',
 'sofa',
 'sirs',
 'apsiii',
 'sapsii',
 'oasis']]

# 结局

In [55]:
outcomes = data[['los_icu',
'los_hospital',
'hospital_expire_flag',
 'stay_chart_included_mdr',
 'stay_chart_included_mdr_neg_tot',
 'stay_chart_included_mdr_pos_tot']]

# 汇总

In [56]:
cohort_total = pd.concat([basic_inf,antsel_sel_df,labvitalssel_sel_df,admissiontype_sel_df,history_sel_df,comb_sel_df,therapy_inf,scores,outcomes],axis = 1)

# 与实际时间整合

In [57]:
def get_data_MIMIC(sql):
    conn = psycopg2.connect(database="mimiciv2_2",user="postgres",
                           password="postgres",
                            host="10.245.99.98",
                            #ost="172.16.2.76",
                            port="5433")
    cur = conn.cursor()
    try:
        cur.execute(sql)
        #获取表的所有字段名称
        coloumns = [row[0] for row in cur.description]
        result = [[str(item) for item in row] for row in cur.fetchall()]
        return pd.DataFrame(result,columns=coloumns)
    except Exception as ex:
        print(ex)
    finally:
        conn.close()

In [58]:
to_cal_actual_year_df = get_data_MIMIC(
'''select  pat.subject_id,hadm_id,stay_id, anchor_year, anchor_year_group, intime
from mimiciv_hosp.patients pat
inner join mimiciv_icu.icustays ide
on pat.subject_id = ide.subject_id
''')

In [59]:
to_cal_actual_year_df['anchor_year_start'] = to_cal_actual_year_df['anchor_year_group'].apply(lambda x: int(x.split(' - ')[0]))
to_cal_actual_year_df['anchor_year_end'] = to_cal_actual_year_df['anchor_year_group'].apply(lambda x: int(x.split(' - ')[1]))

to_cal_actual_year_df['admityear'] = pd.to_datetime(to_cal_actual_year_df['intime']).apply(lambda x: x.year)

In [60]:
to_cal_actual_year_df

,subject_id,hadm_id,stay_id,anchor_year,anchor_year_group,intime,anchor_year_start,anchor_year_end,admityear
0,10001884,26184834,37510196,2122,2008 - 2010,2131-01-11 04:20:05,2008,2010,2131
1,10005348,25239799,34629895,2128,2011 - 2013,2130-10-27 12:06:00,2011,2013,2130
2,10005909,20199380,36496303,2144,2014 - 2016,2144-10-29 23:09:03,2014,2016,2144
3,10006580,24159665,38329661,2137,2014 - 2016,2137-08-10 16:23:10,2014,2016,2137
4,10018928,22523751,35050109,2125,2008 - 2010,2125-02-27 10:42:00,2008,2010,2125
...,...,...,...,...,...,...,...,...,...
73176,19991135,24563775,33094484,2124,2008 - 2010,2133-02-06 10:02:26,2008,2010,2133
73177,19991135,29872770,33242276,2124,2008 - 2010,2133-07-07 08:15:48,2008,2010,2133
73178,19992202,20329411,36409124,2153,2014 - 2016,2153-02-26 20:47:12,2014,2016,2153
73179,19997448,23560173,33380030,2121,2014 - 2016,2122-07-23 10:35:21,2014,2016,2122


In [61]:
to_cal_actual_year_df['actual_year_start'] = to_cal_actual_year_df['admityear'] -  \
                                             to_cal_actual_year_df['anchor_year'].astype(int) + to_cal_actual_year_df['anchor_year_start']

to_cal_actual_year_df['actual_year_end'] = to_cal_actual_year_df['admityear'] -  \
                                             to_cal_actual_year_df['anchor_year'].astype(int) + to_cal_actual_year_df['anchor_year_end']

to_cal_actual_year_df['actual_year_group'] = to_cal_actual_year_df['actual_year_start'].astype(str) + ' - '\
                                            + to_cal_actual_year_df['actual_year_end'].astype(str)

In [62]:
actual_year_df = to_cal_actual_year_df[['stay_id','actual_year_group']]

In [63]:
trans_id(actual_year_df)

/tmp/ipykernel_2821004/1820869256.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[i] = df[i].astype(float)


In [64]:
cohort_total_acty = pd.merge(cohort_total,actual_year_df, how = 'left', on = ['stay_id'])

In [65]:
pd.pivot_table(data = cohort_total_acty,values='stay_chart_included_mdr',index = ['actual_year_group'],aggfunc= ['count','mean'])

,count,mean
,stay_chart_included_mdr,stay_chart_included_mdr
actual_year_group,,
2008 - 2010,7279,0.079132
2009 - 2011,2096,0.134065
2010 - 2012,1582,0.127054
2011 - 2013,9297,0.077875
2012 - 2014,2408,0.111711
2013 - 2015,1688,0.096564
2014 - 2016,9453,0.063472
2015 - 2017,2349,0.098765


In [66]:
float_list = [x for x in cohort_total_acty.columns.to_list() if x not in ['gender','race','last_careunit','actual_year_group']]

In [67]:
# 打印出尝试转换的列名
print("Attempting to convert columns to float:")

for col in float_list:
    try:
        # 尝试将该列转换为浮点数
        cohort_total_acty[col] = cohort_total_acty[col].astype(float)
    except ValueError as e:
        # 如果转换失败，打印列名和错误信息
        print(f"Error converting column {col} to float: {e}")

print("Conversion completed.")


Attempting to convert columns to float:
Conversion completed.


In [68]:
labs_var = [
    'albumin', 'globulin', 'total_protein', 'aniongap', 'bicarbonate',
    'bun', 'calcium', 'chloride', 'creatinine', 'glucose', 'sodium',
    'potassium', 'so2', 'po2', 'pco2', 'fio2_chartevents', 'fio2', 'aado2',
    'aado2_calc', 'pao2fio2ratio', 'ph', 'baseexcess', 'totalco2',
    'carboxyhemoglobin', 'methemoglobin', 'lactate', 'specimen_id',
    'basophils_abs', 'eosinophils_abs', 'lymphocytes_abs', 'monocytes_abs',
    'neutrophils_abs', 'basophils', 'eosinophils', 'lymphocytes',
    'monocytes', 'neutrophils', 'atypical_lymphocytes', 'bands',
    'immature_granulocytes', 'metamyelocytes', 'nrbc', 'bleeding_time',
    'd_dimer', 'fibrinogen', 'thrombin', 'inr', 'pt', 'ptt', 'hematocrit',
    'hemoglobin', 'mch', 'mchc', 'mcv', 'platelet', 'rbc', 'rdw', 'wbc',
    'troponin_i', 'troponin_t', 'ck_mb', 'alt', 'alp', 'ast', 'amylase',
    'bilirubin_total', 'bilirubin_direct', 'bilirubin_indirect', 'ck_cpk',
    'ggt', 'ld_ldh', 'heart_rate', 'spo2', 'sbp', 'dbp', 'mbp', 'resp_rate',
    'temperature', 
    'crp', 'ck', 'ntprobnp', 'mg', 'mpv', 'phosphate'
]


In [69]:
(100-pd.isna(cohort_total_acty[labs_var]).sum()*100/data.shape[0]).sort_values()[:60]

mpv                       0.000000
troponin_i                0.000000
thrombin                  0.249727
d_dimer                   0.302191
methemoglobin             0.411315
carboxyhemoglobin         0.478469
ggt                       0.982120
globulin                  1.949551
total_protein             3.200285
crp                       3.330395
aado2                     4.427936
nrbc                      5.389071
bilirubin_indirect        6.110971
bilirubin_direct          6.337614
amylase                   9.185344
fio2                      9.487535
so2                       9.596659
ntprobnp                 13.560816
atypical_lymphocytes     14.668849
metamyelocytes           14.912281
fio2_chartevents         15.313103
bands                    15.925879
pao2fio2ratio            20.765131
aado2_calc               20.765131
immature_granulocytes    23.428188
troponin_t               27.291614
fibrinogen               29.098464
bleeding_time            30.017628
ck_mb               

In [70]:
#于此处查看缺失值

(100-pd.isna(cohort_total_acty[labs_var]).sum()*100/data.shape[0]).sort_values()[60:90]

mg             97.236213
resp_rate      99.582389
dbp            99.599177
sbp            99.611769
mbp            99.725090
aniongap       99.760766
potassium      99.817426
bicarbonate    99.838412
spo2           99.842609
heart_rate     99.861496
rdw            99.867791
mch            99.878284
mchc           99.886678
mcv            99.888777
rbc            99.890876
hemoglobin     99.892974
wbc            99.899270
platelet       99.903467
creatinine     99.911861
bun            99.916058
glucose        99.916058
hematocrit     99.918157
sodium         99.922354
chloride       99.922354
dtype: float64

In [78]:
(100-pd.isna(cohort_total_acty).sum()*100/data.shape[0]).sort_values()[0:50]

mpv                       0.000000
troponin_i                0.000000
thrombin                  0.249727
d_dimer                   0.302191
methemoglobin             0.411315
carboxyhemoglobin         0.478469
ggt                       0.982120
globulin                  1.949551
total_protein             3.200285
crp                       3.330395
aado2                     4.427936
nrbc                      5.389071
bilirubin_indirect        6.110971
bilirubin_direct          6.337614
amylase                   9.185344
fio2                      9.487535
so2                       9.596659
ntprobnp                 13.560816
atypical_lymphocytes     14.668849
metamyelocytes           14.912281
fio2_chartevents         15.313103
bands                    15.925879
pao2fio2ratio            20.765131
aado2_calc               20.765131
immature_granulocytes    23.428188
troponin_t               27.291614
fibrinogen               29.098464
bleeding_time            30.017628
ck_mb               

In [81]:
MISSLOOKUP = cohort_total_acty.copy()

In [82]:
MISSLOOKUP['lymphocytes_abs'] = MISSLOOKUP['lymphocytes_abs'].replace(0, 0.001)
MISSLOOKUP['RPR'] = MISSLOOKUP['rdw']/MISSLOOKUP['platelet']
MISSLOOKUP['NLR'] = MISSLOOKUP['neutrophils_abs']/MISSLOOKUP['lymphocytes_abs']
MISSLOOKUP['shock_index'] = MISSLOOKUP['heart_rate']/MISSLOOKUP['sbp']

In [110]:
(pd.isna(data['height']/(data['weight']*data['weight'])).sum()*100/data.shape[0])

48.493242676068164

In [104]:
(100-pd.isna(data[['weight','height']]).sum()*100/data.shape[0]).sort_values()[0:50]

height    51.727105
weight    98.029464
dtype: float64

In [85]:
labs_co = cohort_total_acty[labs_var]

In [87]:
# 计算每列中缺失值的百分比
missing_percentage = labs_co.isna().mean() * 100

# 找出缺失值大于 30% 的列名
columns_with_missing = missing_percentage[missing_percentage > 30].index.tolist()

print("Columns with more than 30% missing values:", columns_with_missing)

Columns with more than 30% missing values: ['albumin', 'globulin', 'total_protein', 'so2', 'po2', 'pco2', 'fio2_chartevents', 'fio2', 'aado2', 'aado2_calc', 'pao2fio2ratio', 'ph', 'baseexcess', 'totalco2', 'carboxyhemoglobin', 'methemoglobin', 'lactate', 'atypical_lymphocytes', 'bands', 'immature_granulocytes', 'metamyelocytes', 'nrbc', 'bleeding_time', 'd_dimer', 'fibrinogen', 'thrombin', 'troponin_i', 'troponin_t', 'ck_mb', 'alt', 'alp', 'ast', 'amylase', 'bilirubin_total', 'bilirubin_direct', 'bilirubin_indirect', 'ck_cpk', 'ggt', 'ld_ldh', 'crp', 'ck', 'ntprobnp', 'mpv']


In [88]:
labs_use_col = [i for i in labs_co.columns if i not in columns_with_missing]

In [89]:
labs_use_col = ['aniongap',
 'bicarbonate',
 'bun',
 'calcium',
 'chloride',
 'creatinine',
 'glucose',
 'sodium',
 'potassium',
 'basophils_abs',
 'eosinophils_abs',
 'lymphocytes_abs',
 'monocytes_abs',
 'neutrophils_abs',
 'basophils',
 'eosinophils',
 'lymphocytes',
 'monocytes',
 'neutrophils',
 'inr',
 'pt',
 'ptt',
 'hematocrit',
 'hemoglobin',
 'mch',
 'mchc',
 'mcv',
 'platelet',
 'rbc',
 'rdw',
 'wbc',
 'heart_rate',
 'spo2',
 'sbp',
 'dbp',
 'mbp',
 'resp_rate',
 'temperature',
 'mg',
 'phosphate']

In [90]:
['stay_id','calcium',
 'aniongap','potassium', 'bicarbonate',
 'glucose', 'bun', 'sodium', 'chloride',
 'creatinine', 'pco2', 'po2', 'totalco2',
 'baseexcess', 'ph', 'lactate', 'eosinophils',
 'monocytes_abs', 'neutrophils_abs', 'basophils',
 'monocytes', 'neutrophils', 'eosinophils_abs',
 'basophils_abs', 'lymphocytes',
 'lymphocytes_abs', 'ptt', 'pt', 'inr', 'hematocrit',
 'hemoglobin', 'mch', 'mchc', 'mcv', 'platelet',
 'rbc', 'rdw', 'wbc', 'heart_rate', 'spo2',
 'sbp', 'dbp', 'mbp', 'resp_rate', 'temperature']

['stay_id',
 'calcium',
 'aniongap',
 'potassium',
 'bicarbonate',
 'glucose',
 'bun',
 'sodium',
 'chloride',
 'creatinine',
 'pco2',
 'po2',
 'totalco2',
 'baseexcess',
 'ph',
 'lactate',
 'eosinophils',
 'monocytes_abs',
 'neutrophils_abs',
 'basophils',
 'monocytes',
 'neutrophils',
 'eosinophils_abs',
 'basophils_abs',
 'lymphocytes',
 'lymphocytes_abs',
 'ptt',
 'pt',
 'inr',
 'hematocrit',
 'hemoglobin',
 'mch',
 'mchc',
 'mcv',
 'platelet',
 'rbc',
 'rdw',
 'wbc',
 'heart_rate',
 'spo2',
 'sbp',
 'dbp',
 'mbp',
 'resp_rate',
 'temperature']

In [91]:
from collections import Counter

# 假设 list1 和 list2 是你要比较的两个列表
list1 = labs_use_col
list2 = ['calcium',
 'aniongap','potassium', 'bicarbonate',
 'glucose', 'bun', 'sodium', 'chloride',
 'creatinine', 'pco2', 'po2', 'totalco2',
 'baseexcess', 'ph', 'lactate', 'eosinophils',
 'monocytes_abs', 'neutrophils_abs', 'basophils',
 'monocytes', 'neutrophils', 'eosinophils_abs',
 'basophils_abs', 'lymphocytes',
 'lymphocytes_abs', 'ptt', 'pt', 'inr', 'hematocrit',
 'hemoglobin', 'mch', 'mchc', 'mcv', 'platelet',
 'rbc', 'rdw', 'wbc', 'heart_rate', 'spo2',
 'sbp', 'dbp', 'mbp', 'resp_rate', 'temperature']

# 使用 Counter 统计元素出现的频率
counter1 = Counter(list1)
counter2 = Counter(list2)

# 找出相同的元素（交集）
common_elements = list((counter1 & counter2).elements())

# 找出仅在 list1 中的元素（差集）
unique_to_list1 = list((counter1 - counter2).elements())

# 找出仅在 list2 中的元素（差集）
unique_to_list2 = list((counter2 - counter1).elements())

# 打印结果
print("Common elements:", common_elements)
print("Unique to list1:", unique_to_list1)
print("Unique to list2:", unique_to_list2)

Common elements: ['aniongap', 'bicarbonate', 'bun', 'calcium', 'chloride', 'creatinine', 'glucose', 'sodium', 'potassium', 'basophils_abs', 'eosinophils_abs', 'lymphocytes_abs', 'monocytes_abs', 'neutrophils_abs', 'basophils', 'eosinophils', 'lymphocytes', 'monocytes', 'neutrophils', 'inr', 'pt', 'ptt', 'hematocrit', 'hemoglobin', 'mch', 'mchc', 'mcv', 'platelet', 'rbc', 'rdw', 'wbc', 'heart_rate', 'spo2', 'sbp', 'dbp', 'mbp', 'resp_rate', 'temperature']
Unique to list1: ['mg', 'phosphate']
Unique to list2: ['pco2', 'po2', 'totalco2', 'baseexcess', 'ph', 'lactate']


In [95]:
cohort_total_actydropcol = cohort_total_acty.drop(columns = columns_with_missing)#.drop(columns = unique_to_list2)

In [96]:
lab = cohort_total_actydropcol[['stay_id','aniongap', 'bicarbonate', 'bun', 'calcium', \
                   'chloride', 'creatinine', 'glucose', 'sodium', 'potassium',\
                   'basophils_abs', 'eosinophils_abs', 'lymphocytes_abs', 'monocytes_abs',\
                   'neutrophils_abs', 'basophils', 'eosinophils', 'lymphocytes', 'monocytes',\
                   'neutrophils', 'inr', 'pt', 'ptt', 'hematocrit', 'hemoglobin',\
                   'mch', 'mchc', 'mcv', 'platelet', 'rbc', 'rdw', 'wbc',\
                   'heart_rate', 'spo2', 'sbp', 'dbp', 'mbp', 'resp_rate', 'temperature','mg', 'phosphate']]

In [97]:
#mdro_tomice = pd.read_csv('mdro_tomice.csv')

In [98]:
len(lab[pd.isna(lab['wbc'])].columns)

41

In [101]:
missing_percent = lab.isnull().mean(axis=1)
lab_filtered = lab[missing_percent < 0.2]
lab_filtered = lab_filtered[~pd.isna(lab_filtered['wbc'])]

In [102]:
lab.shape

(47652, 41)

In [103]:
lab_filtered.shape

(34923, 41)

In [110]:
34923-23506-8145

3272

In [104]:
cohort_total_actyfilter = cohort_total_actydropcol[cohort_total_actydropcol['stay_id'].isin(lab_filtered.stay_id.to_list())]

In [111]:
cohort_total_actyfilter.to_csv('mdro_tomice_revise.csv',index = False)

In [ ]:
#lab_filtered[lab_filtered['stay_id'] == 36515454]

In [ ]:
data[data['stay_id'] == 36515454]

In [105]:
act_year_ind = pd.pivot_table(data = cohort_total_actyfilter,values='stay_chart_included_mdr',\
               index = ['actual_year_group'],aggfunc= ['count','mean']).index.to_list()
act_year_pre = act_year_ind[0:7]
act_year_lat = act_year_ind[-4:]

In [106]:
train_df = cohort_total_actyfilter[cohort_total_actyfilter['actual_year_group'].isin(act_year_pre)]
val_df = cohort_total_actyfilter[cohort_total_actyfilter['actual_year_group'].isin(act_year_lat)]

In [108]:
train_df.shape

(23506, 167)

In [109]:
val_df.shape

(8145, 167)

In [ ]:
# def lookup_miss(df):
#     nadf = pd.DataFrame(columns=['nan_nums','nan_ratios'])
#     for i in df.columns:
#         nadf.loc[i,'nan_nums'] = pd.isna(df[i]).sum()
#         nadf.loc[i,'nan_ratios'] = (nadf.loc[i,'nan_nums']/df.shape[0])*100
#     return nadf.sort_values(by = 'nan_ratios', ascending = False)